# Correlation Power Analysis (Brier et al. 2004)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import lascar
import numpy as np
import plotly.graph_objects as pgo

from securec.capture import capture

## Exercise 1

In [ ]:
data = capture(
    platform="elmo",
    number_of_traces=500,
    number_of_samples=0,
    fromfile=os.path.abspath("../lecture_3/sbox_lookup.c"),
    inputfunction=lambda _: [random.randint(0, 255)] + 15 * [0],
)

In [ ]:
fig = pgo.Figure()
for d in data:
    fig.add_trace(pgo.Scatter(y=d["trace"]))
fig.show()

In [ ]:
groups = {i: [] for i in range(9)}
for d in data:
    groups[lascar.hamming(d["input"][0])].append(d["trace"][12])

for i, g in groups.items(): 
    groups[i] = np.mean(g)

In [ ]:
fig = pgo.Figure()
fig.add_trace(pgo.Scatter(x=list(groups.keys()), y=list(groups.values())))
fig.show()

## Exercise 3

In [ ]:
def pearson(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

In [ ]:
def aes_sbox_cpa(
    traces,
    key_byte_index=0,
    trace_point=42,
):
    pearsons = [
        (
            abs(
                pearson(
                    traces["trace"][:, trace_point],
                    [
                        lascar.hamming(lascar.tools.aes.sbox[j ^ guess])
                        for j in traces["input"][:, 0]
                    ],
                )
            ),
            guess,
        )
        for guess in range(256)
    ]
    return list(sorted(pearsons, reverse=True))


In [ ]:
aes_sbox_cpa(data, trace_point=21)

## Exercise 4

In [ ]:
import tqdm
def aes_sbox_cpa_2(
    traces,
    key_byte_index=0,
):
    pearsons = []
    for guess in tqdm.tqdm(range(256)):
        pearsons.append(
            np.nanmax([np.abs(pearson(
                    traces["trace"][:, trace_point],
                    [
                        lascar.hamming(lascar.tools.aes.sbox[j ^ guess])
                        for j in traces["input"][:, 0]
                    ],
            )) for trace_point in range(traces["trace"].shape[1])])
        )
    return list(sorted(pearsons, reverse=True))

In [ ]:
aes_sbox_cpa_2(data)

## Exercise 5

In [ ]:
def selection_function(value, guess):
    return lascar.hamming(lascar.tools.aes.sbox[value["input"][0] ^ guess])


trace = lascar.TraceBatchContainer(data["trace"], data)
engine = lascar.CpaEngine(
    "cpa",
    selection_function=selection_function,
    guess_range=range(256),
)

session = lascar.Session(
    trace,
    engine=engine,
    output_method=lascar.TableOutputMethod(engine),
)
session.run(batch_size="auto")